# Stable Diffusion 2 Image Generation Test

This notebook loads `stabilityai/stable-diffusion-2`, configures the DDIM scheduler, and generates images with 500 inference steps.

In [1]:
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler
from matplotlib import pyplot as plt

# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [8]:
# Configuration
model_id = "stabilityai/stable-diffusion-2-1"
# Fallback option if the above fails
# model_id = "stabilityai/stable-diffusion-2-1"

num_inference_steps = 50
guidance_scale = 7.5

# Define your prompts here
prompts = [
    "A professional photograph of an astronaut riding a horse",
    "A hyper-realistic painting of a futuristic city",
    "A cute corgi dog running in a park, high resolution"
]

In [9]:
# Load Model and Scheduler
print(f"Loading model: {model_id}...")
try:
    # Load the pipeline first. This is often more robust.
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id, 
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    )
    
    # Swap scheduler to DDIM
    pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
    pipe = pipe.to(device)
    print("Model loaded successfully. Scheduler swapped to DDIMScheduler.")
    
except Exception as e:
    print(f"Error loading model: {e}")
    print("\nPotential Fixes:")
    print("1. Check your internet connection.")
    print("2. Try using 'stabilityai/stable-diffusion-2-1' instead.")
    print("3. Ensure you have access to the repository (though it should be public).")

Loading model: stabilityai/stable-diffusion-2-1...


Couldn't connect to the Hub: 404 Client Error. (Request ID: Root=1-695b7f90-44e846d90044a35f46f332ec;dd889d66-14fc-4fba-9b91-97c2581c87e0)

Repository Not Found for url: https://huggingface.co/api/models/stabilityai/stable-diffusion-2-1.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated..
Will try to load from local cache.


Error loading model: Cannot find an appropriate cached snapshot folder for the specified revision on the local disk and outgoing traffic has been disabled. To enable repo look-ups and downloads online, pass 'local_files_only=False' as input.

Potential Fixes:
1. Check your internet connection.
2. Try using 'stabilityai/stable-diffusion-2-1' instead.
3. Ensure you have access to the repository (though it should be public).


In [11]:
# Generation Loop
if 'pipe' in locals():
    images = []
    generator = torch.Generator(device).manual_seed(42) # Fixed seed

    for i, prompt in enumerate(prompts):
        print(f"Generating image {i+1}/{len(prompts)} for prompt: '{prompt}'")
        with torch.autocast("cuda"):
            image = pipe(
                prompt, 
                num_inference_steps=num_inference_steps, 
                guidance_scale=guidance_scale,
                generator=generator
            ).images[0]
        images.append(image)
        
        # Optional: Save image
        image.save(f"sd2_generated_{i}.png")

    print("Generation complete.")
else:
    print("Pipeline not loaded, skipping generation.")

Pipeline not loaded, skipping generation.


In [12]:
# Visualize Results
if 'images' in locals() and images:
    fig, axes = plt.subplots(1, len(images), figsize=(20, 5))
    if len(images) == 1:
        axes = [axes]

    for ax, img, prompt in zip(axes, images, prompts):
        ax.imshow(img)
        ax.set_title(prompt[:30] + "..." if len(prompt) > 30 else prompt, fontsize=10)
        ax.axis("off")

    plt.tight_layout()
    plt.show()